In [16]:
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

# 0. Read csv and clean data

In [17]:
df = pd.read_csv("./EEG_data.csv")
df

,SubjectID,VideoID,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel,user-definedlabeln
0,0.0,0.0,56.0,43.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0,0.0
1,0.0,0.0,40.0,35.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0,0.0
2,0.0,0.0,47.0,48.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0,0.0
3,0.0,0.0,47.0,57.0,-5.0,2012240.0,129350.0,61236.0,17084.0,11488.0,62462.0,49960.0,33932.0,0.0,0.0
4,0.0,0.0,44.0,53.0,-8.0,1005145.0,354328.0,37102.0,88881.0,45307.0,99603.0,44790.0,29749.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12806,9.0,9.0,64.0,38.0,-39.0,127574.0,9951.0,709.0,21732.0,3872.0,39728.0,2598.0,960.0,1.0,0.0
12807,9.0,9.0,61.0,35.0,-275.0,323061.0,797464.0,153171.0,145805.0,39829.0,571280.0,36574.0,10010.0,1.0,0.0
12808,9.0,9.0,60.0,29.0,-426.0,680989.0,154296.0,40068.0,39122.0,10966.0,26975.0,20427.0,2024.0,1.0,0.0
12809,9.0,9.0,60.0,29.0,-84.0,366269.0,27346.0,11444.0,9932.0,1939.0,3283.0,12323.0,1764.0,1.0,0.0


### Cleaning

In [18]:
df = df.drop(["SubjectID", "VideoID", "user-definedlabeln"], axis=1)
df = df[df["predefinedlabel"].isin([0, 1])]
df

,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,56.0,43.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0
1,40.0,35.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0
2,47.0,48.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0
3,47.0,57.0,-5.0,2012240.0,129350.0,61236.0,17084.0,11488.0,62462.0,49960.0,33932.0,0.0
4,44.0,53.0,-8.0,1005145.0,354328.0,37102.0,88881.0,45307.0,99603.0,44790.0,29749.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12806,64.0,38.0,-39.0,127574.0,9951.0,709.0,21732.0,3872.0,39728.0,2598.0,960.0,1.0
12807,61.0,35.0,-275.0,323061.0,797464.0,153171.0,145805.0,39829.0,571280.0,36574.0,10010.0,1.0
12808,60.0,29.0,-426.0,680989.0,154296.0,40068.0,39122.0,10966.0,26975.0,20427.0,2024.0,1.0
12809,60.0,29.0,-84.0,366269.0,27346.0,11444.0,9932.0,1939.0,3283.0,12323.0,1764.0,1.0


### Preprocessing

In [28]:
# Train : Test = 8:2
n_objects = df.shape[0]
n_train = int(n_objects * 0.8)
df_train = df.iloc[:n_train]
df_test = df.iloc[n_train:]
df_train

,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,56.0,43.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0
1,40.0,35.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0
2,47.0,48.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0
3,47.0,57.0,-5.0,2012240.0,129350.0,61236.0,17084.0,11488.0,62462.0,49960.0,33932.0,0.0
4,44.0,53.0,-8.0,1005145.0,354328.0,37102.0,88881.0,45307.0,99603.0,44790.0,29749.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10243,75.0,48.0,40.0,4921.0,4488.0,5847.0,1986.0,11964.0,18952.0,12610.0,12316.0,1.0
10244,80.0,53.0,-35.0,7568.0,15851.0,3968.0,6428.0,12973.0,14491.0,6456.0,5664.0,1.0
10245,83.0,43.0,-25.0,7569.0,16395.0,5526.0,5049.0,17977.0,15359.0,12651.0,6388.0,1.0
10246,80.0,41.0,9.0,7185.0,15162.0,2752.0,4574.0,9879.0,26297.0,3494.0,6677.0,1.0


In [37]:
def encode(df_to_encode):
    le = preprocessing.LabelEncoder()

    df_encoded = df_to_encode.copy()
    df_encoded["Attention"] = le.fit_transform(df_encoded["Attention"])
    df_encoded["Mediation"] = le.fit_transform(df_encoded["Mediation"])
    df_encoded["Delta"] = le.fit_transform(df_encoded["Delta"])
    df_encoded["Theta"] = le.fit_transform(df_encoded["Theta"])
    df_encoded["Alpha1"] = le.fit_transform(df_encoded["Alpha1"])
    df_encoded["Alpha2"] = le.fit_transform(df_encoded["Alpha2"])
    df_encoded["Beta1"] = le.fit_transform(df_encoded["Beta1"])
    df_encoded["Beta2"] = le.fit_transform(df_encoded["Beta2"])
    df_encoded["Gamma1"] = le.fit_transform(df_encoded["Gamma1"])
    df_encoded["Gamma2"] = le.fit_transform(df_encoded["Gamma2"])
    df_encoded["Raw"] = le.fit_transform(df_encoded["Raw"])
    df_encoded["predefinedlabel"] = le.fit_transform(df_encoded["predefinedlabel"])

    x_encoded = df_encoded[["Attention", "Mediation", "Raw", "Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]]
    y_encoded = df_encoded["predefinedlabel"]
    
    return (x_encoded, y_encoded)

# 1. KNN

In [38]:
x_train, y_train = encode(df_train)
x_test, y_test = encode(df_test)

clf = KNeighborsClassifier()        # n_neighbors=5
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Predicted: [1 1 0 ... 0 1 0]
Accuracy: 0.5009754194303551
